In [48]:
#Importing libraries
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re
import json
import pandas as pd

# My code
from main import my_build_model, my_train

# HyperParameters:
embed_dim = 14
lstm_out = 1
batch_size = 32
num_words = 2500
# =============


In [3]:
## Preparing dataset

def convert(x):
    """
    Coverting JSON to pandas dataframe

    """    
    ob = json.loads(x)
    for k, v in ob.items():
        if isinstance(v, list):
            ob[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                ob['%s_%s' % (k, kk)] = vv
            del ob[k]
    return ob



def filter_data(data):
    """
    Converting into pandas dataframe and filtering only text and ratings given by the users
    """

    df = pd.DataFrame([convert(line) for line in data])
    df.drop(columns=df.columns.difference(['text','stars']),inplace=True)
    df.loc[:, ("sentiment")] = 0
    

#     #I have considered a rating above 3 as positive and less than or equal to 3 as negative.
    df.loc[:,'sentiment']=['pos' if (x>3) else 'neg' for x in df.loc[:, 'stars']]
    df.loc[:,'text'] = df.loc[:,'text'].apply(lambda x: x.lower())
    df.loc[:,'text'] = df.loc[:,'text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    for idx,row in df.iterrows():
        df.loc[:,'text']= [x for x in df.loc[:,'text']]
    return df


In [44]:
##Buidling the LSTM network using Keras

def build_model(X):
    model = Sequential()
    print(X.shape[1])
    model.add(Embedding(num_words, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

    return model


def train(model):

    Y = pd.get_dummies(data['sentiment']).values
    X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, test_size = 0.20, random_state = 36)

    #Here we train the Network.
    model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,  verbose=5)

    # Measuring score and accuracy on validation set
    score,acc = model.evaluate(X_valid, Y_valid, verbose=2, batch_size=batch_size)
    print("Logloss score: %.2f" % (score))
    print("Validation set Accuracy: %.2f" % (acc))

In [29]:
json_filename = 'review_mockup.json'
with open(json_filename,'rb') as f:
    data = f.readlines()
data = filter_data(data)


In [42]:
tokenizer = Tokenizer(num_words = num_words, split=' ')
tokenizer.fit_on_texts(data.loc[:,'text'].values)
# #print(tokenizer.word_index)  # To see the dicstionary
# X = tokenizer.texts_to_sequences(data.loc[:,'text'].values)
# X = pad_sequences(X)
# print((X[0]))

test=tokenizer.texts_to_sequences(["It was quite awesome and I know why people enjoy it I was also", "Yes, and Yes again and even more really nice than I expected keep doing more like this"])
test = pad_sequences(test)
print(test)


[[  0   0   0   9   6 324 185   2   4 131 289 109 293   9   4   6  63]
 [583   2 583 119   2  84  67  60  74  97   4 667 388 554  67  46  15]]


In [49]:
model = build_model(test)
print(test)
for layer in model.layers:
    print(layer)
print(model.layers[1].weights)

17
[[  0   0   0   9   6 324 185   2   4 131 289 109 293   9   4   6  63]
 [583   2 583 119   2  84  67  60  74  97   4 667 388 554  67  46  15]]
[<tf.Variable 'lstm_31/lstm_cell_31/kernel:0' shape=(14, 4) dtype=float32, numpy=
array([[-0.21732277,  0.3768264 ,  0.21355677, -0.56960326],
       [ 0.3319708 , -0.12626523, -0.23944962,  0.5683081 ],
       [ 0.5289732 ,  0.06493247, -0.38721386, -0.42209256],
       [-0.424585  ,  0.04019493,  0.08187032,  0.01120645],
       [-0.02018017, -0.39744174, -0.14973843, -0.00911444],
       [ 0.0773834 , -0.33750904, -0.36691898,  0.5760077 ],
       [-0.15068161, -0.13939714,  0.31443846,  0.21029115],
       [-0.1162456 ,  0.3280173 ,  0.2064622 ,  0.2701525 ],
       [ 0.06009567, -0.4509277 ,  0.17181736,  0.5647385 ],
       [ 0.36992615,  0.23637134,  0.52869403, -0.5508219 ],
       [ 0.5034342 ,  0.17473668, -0.21173069, -0.57606214],
       [ 0.4863819 , -0.13593066,  0.31461012,  0.47936463],
       [ 0.08673334,  0.33872217, -0.082

In [15]:
train(model)

NameError: name 'data' is not defined

In [ ]:
y_prob = model.predict(test) 
y_classes = y_prob.argmax(axis=-1)
print(y_prob)
print(y_classes)



In [8]:
############### My code
model = my_build_model(X, hidden_size=lstm_out, input_size = len(X[0]))
Y = pd.get_dummies(data['sentiment']).values
print(Y)


[[ True False]
 [False  True]
 [ True False]
 ...
 [ True False]
 [False  True]
 [ True False]]


In [9]:
print(len(X[0]))
my_train(model, X, Y, lstm_out, len(X[0]))

944
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
  

NameError: name 'exit' is not defined